In [ ]:
#default_exp data

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
from datasets import load_dataset, load_metric, load_from_disk, DatasetDict
import torch
from travis_attack.trainer import get_vm_probs
from travis_attack.config import Config
from fastcore.meta import delegates

# dev
import inspect


# Preparing data

## Classes 

### Base class

In [ ]:
#export
class _BaseDataset(): 
    """Common functionality between datasets. 
    Base class that wraps a huggingface dataset."""
    def __init__(self, cfg): 
        self.cfg = cfg
        
    def preprocess_dataset(self): 
        dsd = self.dsd_raw.map(self._add_idx, batched=True, with_indices=True)
        if self.cfg.use_small_ds: dsd = self.prep_small_ds(dsd)  # do after adding idx so it's consistent across runs
        dsd = dsd.map(self._get_vm_orig_score, batched=True)
        if self.cfg.remove_misclassified_examples: dsd = dsd.filter(lambda x: x['orig_vm_predclass'] == x['label'])
        dsd = dsd.map(self._get_sts_orig_embeddings, batched=True)
        dsd = dsd.map(self.tokenize_fn,             batched=True) 
        dsd = dsd.map(self.add_n_tokens,            batched=True)
        if bucket_by_length: dsd = dsd.sort("n_tokens", reverse=True)
        self.dld_raw = get_dataloaders_dict(dsd, collate_fn=collate_fn_raw) 
        self.dld_tkn = get_dataloaders_dict(dsd, collate_fn=collate_fn_tkn)
        
    def _add_idx(self, x, idx):
        """Add row numbers"""
        x['idx'] = idx; 
        return x   
    
    def _add_n_tokens(self, x): 
        """Add the number of tokens of the "text" field"""
        x['n_tokens'] = [len(o) for o in x['input_ids']]
        return x 
    
    def _get_sts_orig_embeddings(self, sts_model, batch): 
        batch['orig_sts_embeddings'] = sts_model.encode(batch['text'], batch_size=64, convert_to_tensor=False)
        return batch
    
    def _get_vm_orig_score(self, batch): 
        labels = torch.tensor(batch['label'], device=self.cfg.device)
        orig_probs,orig_predclass = get_vm_probs(text = batch[self.cfg.orig_cname], return_predclass=True)
        batch['orig_truelabel_probs'] = torch.gather(orig_probs,1, labels[:,None]).squeeze().cpu().tolist()
        batch['orig_vm_predclass'] = orig_predclass.cpu().tolist()
        return batch
    
    def tokenize_fn(self, x):  
        return pp_tokenizer(x['text'], truncation=True, max_length=max_length)  
    
    def collate_fn_tkn(self, x): 
        d = dict()
        for k in ['idx', 'attention_mask', 'input_ids', 'label', 'orig_truelabel_probs', 'orig_sts_embeddings']: 
            d[k] = [o[k] for o in x]
        return pp_tokenizer.pad(d, pad_to_multiple_of=padding_multiple, return_tensors="pt")

    def collate_fn_raw(self, x): 
        d = dict()
        for k in ['text', 'idx']: 
            d[k] = [o[k] for o in x]
        return d 

    def _get_sampler(self, ds): 
        """Returns a RandomSampler.
        Used so we can keep the same shuffle order across multiple data loaders."""
        g = torch.Generator()
        g.manual_seed(seed)
        return RandomSampler(ds, generator=g)
    
    def prep_small_ds(self, ds_dict):
        """Replaces datasets with a sharded version of themselves."""
        for k,v in ds_dict.items():  
            ds_dict[k] = v.shard(self.cfg.n_shards, 0, contiguous=self.cfg.shard_contiguous)
        return ds_dict
    
    def show_random_elements(self, dataset, num_examples=10):
        """Print some elements in a nice format so you can take a look at them. Use for a dataset from the `datasets` package.  """
        import datasets
        import random
        import pandas as pd
        from IPython.display import display, HTML

        assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
        picks = []
        for _ in range(num_examples):
            pick = random.randint(0, len(dataset)-1)
            while pick in picks:
                pick = random.randint(0, len(dataset)-1)
            picks.append(pick)

        df = pd.DataFrame(dataset[picks])
        for column, typ in dataset.features.items():
            if isinstance(typ, datasets.ClassLabel):
                df[column] = df[column].transform(lambda i: typ.names[i])
        display(HTML(df.to_html()))
        
    def get_dataloaders_dict(self, ds_dict, collate_fn): 
        if bucket_by_length and shuffle_train:  raise Exception("Can only do one of bucket by length or shuffle")
        d = dict()
        for split, ds in ds_dict.items(): 
            if shuffle_train:
                if split == "train": 
                    sampler = get_sampler(ds)
                    d[split] =  DataLoader(ds, batch_size=batch_size_train, sampler=sampler, collate_fn=collate_fn, 
                                           num_workers=n_wkrs, pin_memory=pin_memory) 
                else: 
                    d[split] =  DataLoader(ds, batch_size=batch_size_eval, shuffle=False, collate_fn=collate_fn, 
                                           num_workers=n_wkrs, pin_memory=pin_memory) 
            if bucket_by_length: 
                batch_size = batch_size_train if split == "train" else batch_size_eval
                d[split] =  DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, 
                                       num_workers=n_wkrs, pin_memory=pin_memory) 

        # Add eval dataloader for train 
        d['train_eval'] = DataLoader(ds_dict['train'], batch_size=batch_size_eval, shuffle=False,
                                    collate_fn=collate_fn, num_workers=n_wkrs, pin_memory=pin_memory) 
        return d 
    
    

### Rotten Tomatoes dataset 

In [ ]:
#export
#@delegates()
class RottenTomatoesDataset(_BaseDataset): 
    def __init__(self, cfg): 
        super().__init__(cfg)
        self.prep_dsd_raw()
        self.preprocess_dataset()
        
   
    def prep_dsd_raw(self):
        """Load raw data and package it up in a DatasetDict (dsd) with splits for train, valid, test."""
        self.dsd_raw = load_dataset("rotten_tomatoes")
        self.dsd_raw['valid'] = self.dsd_raw.pop('validation')  # "valid" is easier than "validation"
        for _,ds in self.dsd_raw.items():
            # make sure that all datasets have the same number of labels as what the victim model predicts
            assert ds.features[self.cfg.label_cname].num_classes == self.cfg.vm_num_labels 
    

### Simple dataset 

In [ ]:
#export
class SimpleDatasetWrapper(_BaseDataset): 
    def __init__(self, path_data, splits): 
        super().__init__()
        dsd_raw = DatasetDict()
        for s in splits:  
            dsd_raw[s] = load_dataset('csv', data_files=f"{path_data}simple_dataset_{s}.csv")['train']

In [ ]:
cfg = Config().rotten_tomatoes_dataset()
cfg.vm_num_labels = 2  # replace this with code that determines it later


# load vm 
# load sts model 


ds = RottenTomatoesDataset(cfg=cfg)


Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/data/tproth/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
ds.dsd_raw

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
inspect.signature(RottenTomatoesDataset)

<Signature (cfg)>

### SNLI dataset (not implemented )

In [ ]:
class SNLIDatasetWrapper(_DatasetWrapper): 
    def __init__(): 
        super().__init__()
        pass
        ## For snli
        # remove_minus1_labels = lambda x: x[label_cname] != -1
        # ds_train = ds_train.filter(remove_minus1_labels)
        # valid = valid.filter(remove_minus1_labels)
        # test = test.filter(remove_minus1_labels)

# Usage

In [ ]:
cfg

## Functions that add new fields to the dataset

## Compute VM and STS scores for a text

## Functions used to prepare dataloaders

## Misc functions 

## Export 

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()